# **Use-Case**
This project aims to optimize a portfolio of stocks to achieve the maximum return for a given level of risk. Using historical stock data, you will apply the Markowitz model (mean-variance optimization) to determine the ideal asset allocation for the portfolio

**Tasks**:
1. Data Collection
  - Select 10 Stocks from different sectors
  - Pull historical data (daily closing prices) for each stock over the past 3 years using Yahoo Finance
  - Plot the stock prices for each stock over time
2. Data Preprocessing and Feature Engineering
  - Calculate daily and annual returns for each stock
  - Calculate covariance and correlation between stock returns to understand the diversification benefits
  - Plot the Correlationmatrix
3. Exploratory Data Analysis (EDA):
  - Visualize the distribution of returns for each stock
  - Think about some interessing analysis work an plot it.
  - Report Insights and Observations
4. Mean-Variance Optimization
  - Use the Markowitz model to calculate the optimal portfolio allocation
  - Define a risk tolerance level and find the asset weights that maximize returns for that risk level
  - Visualize the efficient frontier and identify the optimal portfolio on the curve

**Summarize**:
  - Summarize findings in a report, including the chosen portfolio allocation, expected return, and risk characteristics

# **Summary**
In the analysis, three optimized portfolios were identified along the efficient frontier, each tailored to different investment goals: maximizing the Sharpe Ratio, meeting a specific risk tolerance, and minimizing volatility. Here’s a breakdown of each:

    Optimal Portfolio by Highest Sharpe Ratio:
        Expected Return: 18.79%
        Volatility: 17.15%
        Key Holdings: This portfolio is heavily weighted in XOM (48.06%) and includes notable positions in AAPL (16.5%) and PG (16.81%).
        Sharpe Ratio: 1.04
        Characteristics: This portfolio maximizes the risk-adjusted return, aiming for a high return while maintaining a moderate level of risk.

    Portfolio with Risk Tolerance of 15% Volatility:
        Expected Return: 15.77%
        Volatility: 14.77%
        Key Holdings: Significant allocations in XOM (33.78%), PG (28.6%), and AAPL (11.28%).
        Characteristics: This portfolio targets an acceptable return within a risk threshold of 15% volatility, balancing return potential with controlled risk exposure.

    Portfolio with Lowest Volatility:
        Expected Return: 8.42%
        Volatility: 12.48%
        Key Holdings: High allocation in JNJ (32.59%), PG (21.41%), and VZ (12.18%).
        Characteristics: Designed for risk-averse investors, this portfolio focuses on minimizing volatility, resulting in a lower expected return but greater stability.

Conclusion

These portfolios illustrate different risk-return profiles:

- The Highest Sharpe Ratio Portfolio is suitable for investors seeking the best risk-adjusted return.
- The Risk-Tolerance Portfolio is ideal for those with a moderate risk appetite, aiming to maximize returns within a specified risk level.
- The Lowest Volatility Portfolio is tailored for conservative investors prioritizing stability over high returns.

The efficient frontier chart demonstrates the potential for diversification to achieve these tailored portfolios within the defined risk-return framewor


# **Data Collection**

In [38]:
import pandas as pd
import yfinance as yf

def data_collection():
  # Set tickers
  tickers = ['AAPL', 'MSFT', 'JNJ', 'JPM', 'XOM', 'PG', 'VZ', 'UNH', 'HD', 'DIS']

  # Set timeframe
  end_date = pd.Timestamp.today() # get todays date
  start_date = end_date - pd.DateOffset(years=3) # 3 years back

  # Loading prices
  df_stocks = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
  df_stocks = df_stocks.round(2)
  df_stocks.reset_index(inplace=True)

  # Add Weekday, Month, and Year columns using dt accessor
  df_stocks['Weekday'] = df_stocks['Date'].dt.day_name()
  df_stocks['Month'] = df_stocks['Date'].dt.month_name()
  df_stocks['Year'] = df_stocks['Date'].dt.year

  # Convert Date to date format (remove time)
  df_stocks['Date'] = df_stocks['Date'].dt.date

  # Reorder columns to place Date, Weekday, Month, Year at the beginning
  cols = ['Date', 'Weekday', 'Month', 'Year'] + [col for col in df_stocks.columns if col not in ['Date', 'Weekday', 'Month', 'Year']]
  df_stocks = df_stocks[cols]

  return df_stocks, tickers

df_stocks, tickers = data_collection()


[*********************100%***********************]  10 of 10 completed


In [40]:
df_stocks.head()

Ticker,Date,Weekday,Month,Year,AAPL,DIS,HD,JNJ,JPM,MSFT,PG,UNH,VZ,XOM
0,2021-11-08,Monday,November,2021,148.18,175.49,341.83,149.52,155.43,328.53,134.59,443.31,43.33,58.80
1,2021-11-09,Tuesday,November,2021,148.54,173.74,343.74,149.19,154.25,327.51,134.99,444.97,43.26,59.38
2,2021-11-10,Wednesday,November,2021,145.70,173.08,341.82,150.80,154.11,322.49,136.32,440.71,43.56,58.21
3,2021-11-11,Thursday,November,2021,145.65,160.84,340.94,149.69,154.10,324.08,135.93,438.30,43.43,58.32
4,2021-11-12,Friday,November,2021,147.74,158.38,345.58,151.48,153.41,328.26,136.08,439.83,43.34,57.87


## **Plotting Stock Prices**

In [45]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_stock_prices(df_Stocks, tickers):
  # Initialize figure
  fig = go.Figure()

  # Erstellen der Linien für jeden Ticker
  for ticker in tickers:
    fig.add_trace(go.Scatter(x=df_stocks['Date'], y=df_stocks[ticker], mode='lines', name=ticker))

  # Upate layout
  fig.update_layout(
      title="Stock Price Development Over the Last 3 Years",
      xaxis_title="Date",
      yaxis_title="Adjusted Close Price (USD)",
      xaxis=dict(tickangle=45),
      legend=dict(x=0.95, y=0.95),
      template="plotly_white",
      height=600,
      width=900
    )

  fig.show()

plot_stock_prices(df_stocks, tickers)

# **Data Preprocessing**

## **Data Information**

In [53]:
def data_preprocessing(df_stocks):
  summary = df_stocks.describe()
  nan_values = df_stocks.isnull().sum()
  duplicated_values = df_stocks.duplicated().sum()

  return summary, nan_values, duplicated_values

summary, nan_values, duplicated_values = data_preprocessing(df_stocks)

In [54]:
# summary
# nan_values
duplicated_values

0

## **Feature Engineering - Calculating Daily Returns, Covarianz Matrix, Correlation Matrix**

In [57]:
def calc_daily_returns(df_stocks):
  df_stocks['Date'] = pd.to_datetime(df_stocks['Date'])
  df_daily_returns = df_stocks[tickers].pct_change().dropna() * 100
  df_daily_returns.set_index(df_stocks['Date'][1:], inplace=True) # removing first row

  cov_matrix = df_daily_returns.cov()
  corr_matrix = df_daily_returns.corr()

  return df_daily_returns, cov_matrix, corr_matrix

df_daily_returns, cov_matrix, corr_matrix = calc_daily_returns(df_stocks)

In [59]:
# df_daily_returns.head()
# cov_matrix
corr_matrix

Ticker,AAPL,MSFT,JNJ,JPM,XOM,PG,VZ,UNH,HD,DIS
Ticker,,,,,,,,,,
AAPL,1.000000,0.686783,0.176385,0.364477,0.176022,0.274377,0.157247,0.244094,0.468210,0.436556
MSFT,0.686783,1.000000,0.138087,0.333022,0.085480,0.281664,0.085456,0.227879,0.451813,0.442697
JNJ,0.176385,0.138087,1.000000,0.269878,0.109331,0.455810,0.267735,0.353930,0.261959,0.124863
JPM,0.364477,0.333022,0.269878,1.000000,0.322985,0.252109,0.224128,0.252862,0.399876,0.427058
XOM,0.176022,0.085480,0.109331,0.322985,1.000000,0.053933,0.200933,0.182051,0.168997,0.256399
PG,0.274377,0.281664,0.455810,0.252109,0.053933,1.000000,0.320526,0.362220,0.333699,0.180864
VZ,0.157247,0.085456,0.267735,0.224128,0.200933,0.320526,1.000000,0.200644,0.223152,0.206278
UNH,0.244094,0.227879,0.353930,0.252862,0.182051,0.362220,0.200644,1.000000,0.249337,0.152346
HD,0.468210,0.451813,0.261959,0.399876,0.168997,0.333699,0.223152,0.249337,1.000000,0.419920


In [72]:
import plotly.graph_objs as go
import plotly.express as px

def plot_correlation_heatmap(corr_matrix):
    fig = px.imshow(corr_matrix, text_auto='.2f', color_continuous_scale="RdBu", zmin=-1, zmax=1)

    fig.update_layout(
        title="Correlation Matrix of Daily Stock Returns", width=800, height=600)

    fig.update_traces(textfont=dict(size=10))

    fig.show()

plot_correlation_heatmap(corr_matrix)

# **Explorative Data Analysis**

## **Daily Returns Distribution**

In [73]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def daily_return_histograms(df_daily_returns):
  # Create layout for subplots
  rows = (len(tickers) // 3) + (len(tickers) % 3 > 0)
  fig = make_subplots(rows=rows, cols=3, subplot_titles=tickers)

  # Create histogram for every subplot
  for i, ticker in enumerate(tickers):
    row = (i // 3) + 1
    col = (i % 3) + 1
    histogram = go.Histogram(x=df_daily_returns[ticker], nbinsx=50, name=ticker)
    fig.add_trace(histogram, row=row, col=col)

  # Update layout
  fig.update_layout(xaxis_title="Daily Return (%)", yaxis_title="Frequency", showlegend=False, height=800, width=1200,)

  for i in range(1, len(tickers) + 1):
    fig.layout.annotations[i - 1].font.size = 12  # Schriftgröße auf 10 setzen

  for i in range(1, rows * 3 + 1):
    fig.update_xaxes(title_text="Daily Return (%)", row=(i // 3) + 1, col=(i % 3) + 1)
    fig.update_yaxes(title_text="Frequency", row=(i // 3) + 1, col=(i % 3) + 1)

  fig.show()

daily_return_histograms(df_daily_returns)

## **Average Returns by Weekday, Month, Year**

In [74]:
def recalculation(df_daily_returns, df_stocks):
  df_daily_returns['Weekday'] = df_stocks['Weekday'][1:].values
  df_daily_returns['Month'] = df_stocks['Month'][1:].values
  df_daily_returns['Year'] = df_stocks['Year'][1:].values
  cols = ['Weekday', 'Month', 'Year'] + [col for col in df_daily_returns.columns if col not in ['Weekday', 'Month', 'Year']]
  df_daily_returns = df_daily_returns[cols]

  return df_daily_returns

df_daily_returns = recalculation(df_daily_returns, df_stocks)

In [75]:
def create_average_return_plot(df_daily_returns):
  ticker_list = ['AAPL', 'MSFT', 'JNJ', 'JPM', 'XOM', 'PG', 'VZ', 'UNH', 'HD', 'DIS']
  timeframes = { # dictionary for the three timeframes (keys) + tupel of values for timeframes and column name in df_daily_returns for grouping
      'Weekday': (['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], 'Weekday'),
      'Month': (['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], 'Month'),
      'Year': ([2021, 2022, 2023, 2024], 'Year')
  }

  # List for melted dataframes
  melted_df = []

  for timeframe, (order, column) in timeframes.items():
    avg_returns = df_daily_returns.groupby(column)[ticker_list].mean()
    avg_returns = avg_returns.loc[order]
    df_melted = avg_returns.reset_index().melt(id_vars=column, var_name='Ticker', value_name='Average Return')
    melted_df.append(df_melted)

  # Create subplots
  fig = make_subplots(rows=1, cols=3, subplot_titles=["Average Return by Weekday", "Average Return by Month", "Average Return by Year"])

  # Add bar charts for each timeframe
  for i, (df_melted, title) in enumerate(zip(melted_df, ["Weekday", "Month", "Year"])):
        for ticker in df_melted['Ticker'].unique():
          fig.add_trace(
              go.Bar(x=df_melted[df_melted['Ticker'] == ticker][title],
                      y=df_melted[df_melted['Ticker'] == ticker]['Average Return'],
                      name=ticker, showlegend=(i == 0)),
              row=1, col=i+1
            )

  # Update layout
  fig.update_layout(
        height=500,
        width=1500,
        title_text="Average Daily Return by Weekday, Month, and Year",
        legend=dict(x=1.05, y=1)
    )

  # Update x and y axis titles
  fig.update_xaxes(title_text="Weekday", row=1, col=1)
  fig.update_xaxes(title_text="Month", row=1, col=2)
  fig.update_xaxes(title_text="Year", row=1, col=3)
  fig.update_yaxes(title_text="Average Return (%)", row=1, col=1)
  fig.update_yaxes(title_text="Average Return (%)", row=1, col=2)
  fig.update_yaxes(title_text="Average Return (%)", row=1, col=3)

  fig.show()

create_average_return_plot(df_daily_returns)

## **Maximal Daily Loss by Stocks**

In [106]:
def calculate_max_daily_loss(df_daily_returns):
    df_numeric = df_daily_returns.select_dtypes(include=[float])
    max_daily_losses = {}

    for column in df_numeric.columns:
        max_daily_losses[column] = df_numeric[column].min() / 100 # Find the minimum (largest negative return) in each column

    return max_daily_losses

max_daily_losses = calculate_max_daily_loss(df_daily_returns)

In [110]:
def plot_max_daily_loss(max_daily_losses):
    df_max_daily_losses = pd.DataFrame(list(max_daily_losses.items()), columns=['Stock', 'Max Daily Loss']) # Convert max_daily_losses dictionary to a DataFrame for easier plotting

    # Create a Plotly bar chart
    fig = go.Figure(go.Bar(
        x=df_max_daily_losses['Stock'],
        y=df_max_daily_losses['Max Daily Loss'],
        marker_color='indianred'
    ))

    # Customize the layout
    fig.update_layout(
        title="Maximum Daily Loss of Each Stock",
        xaxis_title="Stock",
        yaxis_title="Max Daily Loss (%)",
        yaxis_tickformat=".2%",  # Display as a percentage
        template="plotly_white",
        height=500,
        width=800
    )

    fig.show()

plot_max_daily_loss(max_daily_losses)

## **Daily Volatility by Stock**

In [114]:
def calculate_volatility(df_daily_returns):
    df_numeric = df_daily_returns.select_dtypes(include=[float])
    volatilities = {}

    for column in df_numeric.columns:
        volatilities[column] = df_numeric[column].std() / 100  # Calculate the standard deviation of daily returns and convert to percentage

    return volatilities

volatilities = calculate_volatility(df_daily_returns)

In [116]:
def plot_volatility(volatilities):
    df_volatilities = pd.DataFrame(list(volatilities.items()), columns=['Stock', 'Volatility'])  # Convert volatilities dictionary to a DataFrame for easier plotting

    # Create a Plotly bar chart
    fig = go.Figure(go.Bar(
        x=df_volatilities['Stock'],
        y=df_volatilities['Volatility'],
        marker_color='dodgerblue'
    ))

    # Customize the layout
    fig.update_layout(
        title="Volatility of Each Stock Over the Period",
        xaxis_title="Stock",
        yaxis_title="Volatility (%)",
        yaxis_tickformat=".2%",  # Display as a percentage
        template="plotly_white",
        height=500,
        width=800
    )

    fig.show()

plot_volatility(volatilities)

# **Mean-Variance Optimization**

In [119]:
import scipy.optimize as sco
import numpy as np
df_numeric = df_daily_returns.select_dtypes(include=[float])

## **Calculate Annualized Returns and Covariance Matrix**

In [140]:
def calculate_annualized_returns(df_numeric):
    mean_daily_returns = df_numeric.mean()
    annualized_returns = mean_daily_returns * 252  # Assuming 252 trading days in a year
    return annualized_returns

annualized_returns = calculate_annualized_returns(df_numeric)

In [141]:
def calculate_covariance_matrix(df_numeric):
    covariance_matrix = df_numeric.cov() * 252  # Annualizing the daily covariance matrix
    return covariance_matrix

covariance_matrix = calculate_covariance_matrix(df_numeric)

## **Generating Random Portfolios**

In [142]:
def generate_random_portfolios(annualized_returns, covariance_matrix, num_portfolios=5000):
    num_assets = len(annualized_returns)
    results = {'volatility': [], 'return': []}

    for _ in range(num_portfolios):
        # Generiere zufällige Gewichtungen und normalisiere sie
        weights = np.random.rand(num_assets)
        weights /= np.sum(weights)

        # Berechne die Rendite und Volatilität des Portfolios
        portfolio_return = np.dot(weights, annualized_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

        results['volatility'].append(portfolio_volatility)
        results['return'].append(portfolio_return)

    return results

random_portfolios = generate_random_portfolios(annualized_returns, covariance_matrix)

## **Calculating the Efficient Portfolios (Frontier)**

In [135]:
def calculate_efficient_frontier_with_constraints(annualized_returns, covariance_matrix, num_points=100):
    num_assets = len(annualized_returns)
    frontier_returns = np.linspace(annualized_returns.min(), annualized_returns.max(), num_points)
    frontier_volatility = []

    for ret in frontier_returns:
        def portfolio_volatility(weights):
            return np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

        # Constraints: Weights sum to 1 and target return
        constraints = (
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
            {'type': 'eq', 'fun': lambda weights: np.dot(weights, annualized_returns) - ret}
        )
        # Bounds for weights: 0.02 <= weight <= 1 for each asset (no short positions, minimum 2% allocation)
        bounds = tuple((0.02, 1) for _ in range(num_assets))
        initial_weights = np.array(num_assets * [1.0 / num_assets])

        result = sco.minimize(portfolio_volatility, initial_weights, bounds=bounds, constraints=constraints)

        if result.success:
            frontier_volatility.append(result.fun)
        else:
            # If optimization fails, append NaN for this point
            frontier_volatility.append(np.nan)

    return list(zip(frontier_volatility, frontier_returns))

efficient_frontier = calculate_efficient_frontier_with_constraints(annualized_returns, covariance_matrix)

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:441: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:495: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



## **Plotting the Efficient Frontier**

In [143]:
def plot_efficient_frontier_with_random_portfolios(efficient_frontier, random_portfolios):
    # Filter out any points where optimization failed in the efficient frontier
    filtered_frontier = [(vol, ret) for vol, ret in efficient_frontier if not np.isnan(vol)]
    if not filtered_frontier:
        print("No feasible portfolios found under the given constraints.")
        return

    frontier_volatilities, frontier_returns = zip(*filtered_frontier)

    # Plot random portfolios as scatter points
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=random_portfolios['volatility'],
        y=random_portfolios['return'],
        mode='markers',
        name='Random Portfolios',
        marker=dict(color='lightgray', size=5, opacity=0.5)
    ))

    # Plot efficient frontier as a line
    fig.add_trace(go.Scatter(
        x=frontier_volatilities,
        y=frontier_returns,
        mode='lines',
        name='Efficient Frontier',
        line=dict(color='blue')
    ))

    # Customize layout
    fig.update_layout(
        title="Efficient Frontier with Random Portfolios",
        xaxis_title="Volatility (Risk)",
        yaxis_title="Return",
        template="plotly_white",
        xaxis=dict(range=[0, 30]),  # Adjust x-axis range if needed
        height=500,
        width=800
    )

    fig.show()

plot_efficient_frontier_with_random_portfolios(efficient_frontier, random_portfolios)

## **Best Portfolio out of Frontier by Sharpe Ratio**

In [167]:
def find_optimal_portfolio_on_frontier(efficient_frontier, annualized_returns, covariance_matrix, risk_free_rate=1.0):
    max_sharpe_ratio = -np.inf
    optimal_weights = None
    optimal_return = None
    optimal_volatility = None

    for volatility, target_return in efficient_frontier:
        def portfolio_volatility(weights):
            return np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

        # Constraints: Weights sum to 1 and portfolio return equals target return
        constraints = (
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
            {'type': 'eq', 'fun': lambda weights: np.dot(weights, annualized_returns) - target_return}
        )
        # Bounds for weights: 0.02 <= weight <= 1 for each asset
        bounds = tuple((0.02, 1) for _ in range(len(annualized_returns)))
        initial_weights = np.array(len(annualized_returns) * [1.0 / len(annualized_returns)])

        # Optimize to find the weights for this target return
        result = sco.minimize(portfolio_volatility, initial_weights, bounds=bounds, constraints=constraints)

        if result.success:
            weights = result.x
            # Calculate portfolio return and volatility
            portfolio_return = np.dot(weights, annualized_returns)
            portfolio_volatility = portfolio_volatility(weights)
            # Calculate the Sharpe Ratio
            sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

            # Update if this Sharpe Ratio is the highest found
            if sharpe_ratio > max_sharpe_ratio:
                max_sharpe_ratio = sharpe_ratio
                optimal_weights = weights
                optimal_return = portfolio_return
                optimal_volatility = portfolio_volatility

    # Convert optimal weights to a DataFrame in percentages
    optimal_weights_df = pd.DataFrame.from_dict(
        dict(zip(annualized_returns.index, optimal_weights)),
        orient='index',
        columns=['Weight']
    )
    optimal_weights_df['Weight'] = (optimal_weights_df['Weight'] * 100).round(2).astype(str) + '%'

    # Print the optimal portfolio details
    print("Optimal Portfolio:")
    print(optimal_weights_df)
    print("Expected Return:", optimal_return)
    print("Volatility:", optimal_volatility)
    print("Sharpe-Ratio:", max_sharpe_ratio)

# Beispielaufruf
optimal_portfolio = find_optimal_portfolio_on_frontier(efficient_frontier, annualized_returns, covariance_matrix)



/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:441: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:495: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimal Portfolio:
      Weight
AAPL   16.5%
MSFT    2.0%
JNJ     2.0%
JPM    4.19%
XOM   48.06%
PG    16.81%
VZ      2.0%
UNH    4.45%
HD      2.0%
DIS     2.0%
Expected Return: 18.794948944656163
Volatility: 17.147475533230722
Sharpe-Ratio: 1.0377591097984469


## **Best Portfolio out of Frontier by lowest Volatility**

In [166]:
def find_min_volatility_portfolio_on_frontier(efficient_frontier, annualized_returns, covariance_matrix):
    min_volatility = np.inf
    optimal_weights = None
    optimal_return = None
    optimal_volatility = None

    for volatility, target_return in efficient_frontier:
        def portfolio_volatility(weights):
            return np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

        # Constraints: Weights sum to 1 and portfolio return equals target return
        constraints = (
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
            {'type': 'eq', 'fun': lambda weights: np.dot(weights, annualized_returns) - target_return}
        )
        # Bounds for weights: 0.02 <= weight <= 1 for each asset
        bounds = tuple((0.02, 1) for _ in range(len(annualized_returns)))
        initial_weights = np.array(len(annualized_returns) * [1.0 / len(annualized_returns)])

        # Optimize to find the weights for this target return
        result = sco.minimize(portfolio_volatility, initial_weights, bounds=bounds, constraints=constraints)

        if result.success:
            weights = result.x
            # Calculate portfolio volatility
            portfolio_volatility_value = portfolio_volatility(weights)

            # Update if this volatility is the lowest found
            if portfolio_volatility_value < min_volatility:
                min_volatility = portfolio_volatility_value
                optimal_weights = weights
                optimal_return = np.dot(weights, annualized_returns)
                optimal_volatility = portfolio_volatility_value

    # Convert optimal weights to a DataFrame in percentages
    optimal_weights_df = pd.DataFrame.from_dict(
        dict(zip(annualized_returns.index, optimal_weights)),
        orient='index',
        columns=['Weight']
    )
    optimal_weights_df['Weight'] = (optimal_weights_df['Weight'] * 100).round(2).astype(str) + '%'

    # Print the optimal portfolio details
    print("Portfolio with lowest Volatility:")
    print(optimal_weights_df)
    print("Expected Return:", optimal_return)
    print("Volatility:", optimal_volatility)

min_volatility_portfolio = find_min_volatility_portfolio_on_frontier(efficient_frontier, annualized_returns, covariance_matrix)

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:441: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:495: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Portfolio with lowest Volatility:
      Weight
AAPL    2.0%
MSFT   6.74%
JNJ   32.59%
JPM    3.18%
XOM   12.24%
PG    21.41%
VZ    12.18%
UNH    5.65%
HD      2.0%
DIS     2.0%
Expected Return: 8.421034364944383
Volatility: 12.480592672938421


## **Best Portfolio out of Forntier by given Volatility (Risk-Toleranz)**

In [165]:
import numpy as np
import pandas as pd
import scipy.optimize as sco

def find_max_return_portfolio_under_volatility(efficient_frontier, annualized_returns, covariance_matrix, max_volatility=15.00):
    max_return = -np.inf
    optimal_weights = None
    optimal_return = None
    optimal_volatility = None

    for volatility, target_return in efficient_frontier:
        # Only consider portfolios within the maximum volatility threshold
        if volatility > max_volatility:
            continue

        def portfolio_volatility(weights):
            return np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

        # Constraints: Weights sum to 1 and portfolio return equals target return
        constraints = (
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
            {'type': 'eq', 'fun': lambda weights: np.dot(weights, annualized_returns) - target_return}
        )
        # Bounds for weights: 0.02 <= weight <= 1 for each asset
        bounds = tuple((0.02, 1) for _ in range(len(annualized_returns)))
        initial_weights = np.array(len(annualized_returns) * [1.0 / len(annualized_returns)])

        # Optimize to find the weights for this target return
        result = sco.minimize(portfolio_volatility, initial_weights, bounds=bounds, constraints=constraints)

        if result.success:
            weights = result.x
            # Calculate portfolio return and volatility
            portfolio_return = np.dot(weights, annualized_returns)
            portfolio_volatility_value = portfolio_volatility(weights)

            # Update if this return is the highest found within the volatility threshold
            if portfolio_return > max_return:
                max_return = portfolio_return
                optimal_weights = weights
                optimal_return = portfolio_return
                optimal_volatility = portfolio_volatility_value

    # Check if an optimal portfolio was found
    if optimal_weights is None:
        print(f"No Portfolio for given Risk-Tolerance {max_volatility * 100:.2f}% hat.")
        return None

    # Convert optimal weights to a DataFrame in percentages
    optimal_weights_df = pd.DataFrame.from_dict(
        dict(zip(annualized_returns.index, optimal_weights)),
        orient='index',
        columns=['Weight']
    )
    optimal_weights_df['Weight'] = (optimal_weights_df['Weight'] * 100).round(2).astype(str) + '%'

    # Print the optimal portfolio details
    print(f"Portfolio by given Risk-Tolerance {max_volatility:.2f}%:")
    print(optimal_weights_df)
    print("Expected Return:", optimal_return)
    print("Volatility:", optimal_volatility)

    return {
        'weights': optimal_weights_df,
        'return': optimal_return,
        'volatility': optimal_volatility
    }

# Beispielaufruf
optimal_portfolio = find_max_return_portfolio_under_volatility(efficient_frontier, annualized_returns, covariance_matrix, max_volatility=15.00)


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:441: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:495: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Portfolio by given Risk-Tolerance 15.00%:
      Weight
AAPL  11.28%
MSFT    2.0%
JNJ    3.68%
JPM    6.55%
XOM   33.78%
PG     28.6%
VZ      2.0%
UNH    8.11%
HD      2.0%
DIS     2.0%
Expected Return: 15.769223858886022
Volatility: 14.773183689572745
